## IDEIA:

Deixar o código "array-wise". Isto implicaria em vária mudanças para o código:

- Usar DiffToll para derivação de arrays;
- Trocar integrate.quad() para uma função que lide com arrays;
- Possívelmente mudar outras partes do código como em galerkin_method().

Como isso vai dar um pouco de trabalho e exige tempo, só deixo aqui a ideia anotada em uma branch, "array-wise". Caso, no futuro, eu perceber que esse tipo abordagem para o código é melhor e mais flexível, trazendo menos "dor de cabeça", trabalharei nesta ideia.

Fazer um (ou mais) Benchmarks

Preciso ajustar galerkin_method para o primeiro problema variacional da Lista.

Sugestão: criar uma outra função para o funcional 'a' e o 'f' e utilizar dentro de galerkin_method.

In [3]:
import numpy as np
from scipy import integrate
from scipy.misc import derivative

In [4]:
def trapezoidal_rule(func, x_inicial, x_final, num_points):
    x = np.linspace(x_inicial, x_final, num_points)
    y = func(x)
    h = (x_final - x_inicial) / (num_points - 1)
    integral = (h / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])
    return integral

In [5]:
def gaussian_quad(func, x_inicial, x_final, args=None):
    def_integral = integrate.quad(func, x_inicial, x_final)[0]
    return def_integral

In [18]:
def galerkin_method(basis_functions, f, x_inicial, x_final, integration_method, args=None):
    n = len(basis_functions)
    A = np.zeros((n, n))
    l = np.zeros(n)
    h = 1.0 / num_partitions

    for i in range(n):
        for j in range(n):
            integrand = lambda x: derivative(basis_functions[i], x, dx=1e-4) * derivative(basis_functions[j], x, dx=1e-4)
            A[i, j] = integration_method(integrand, x_inicial, x_final, args)
        
        integrand_l = lambda x: basis_functions[i](x) * f(x)
        l[i] = integration_method(integrand_l, x_inicial, x_final, args)

    coef = np.linalg.solve(A, l)
    return coef

In [7]:
def basis_func_1(x):
    return x

def basis_func_2(x):
    return x**2

def f(x):
    return np.sin(x)

basis_functions = [basis_func_1, basis_func_2]
x_inicial = 0
x_final = np.pi
num_partitions = 10

# Using Gaussian quadrature
coef_gaussian = galerkin_method(basis_functions, f, x_inicial, x_final, gaussian_quad)
print("Coefficients with Gaussian quadrature:", coef_gaussian)

# Using Trapezoidal rule
coef_trapezoidal = galerkin_method(basis_functions, f, x_inicial, x_final, trapezoidal_rule, num_partitions)
print("Coefficients with Trapezoidal rule:", coef_trapezoidal)

Coefficients with Gaussian quadrature: [ 2.2158542  -0.38701841]
Coefficients with Trapezoidal rule: [ 2.1763078  -0.37766908]


/tmp/ipykernel_9153/2105876560.py:9: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  integrand = lambda x: derivative(basis_functions[i], x, dx=1e-6) * derivative(basis_functions[j], x, dx=1e-6)


In [33]:
derivada_de_x = lambda x: derivative(basis_func_2,1,dx=1)

In [34]:
derivada_de_x(1)

/tmp/ipykernel_9153/842386134.py:1: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  derivada_de_x = lambda x: derivative(basis_func_2,1,dx=1)


2.0